## Cell image analysis with python via google colab

First we need to set up for colab to be able to find files on google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

We will use the skimage python package for loading image files

In [ ]:
import skimage.io

In [ ]:
img = skimage.io.imread('/content/drive/MyDrive/images/cell_image.jpg') #simply change the file name to analyze a different image

Let's take a quick look at the image using matplotlib's pyplot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.imshow(img)

What data is stored in this img variable?

In [ ]:
print(img.shape)
print(img[0,0,:])

This is a 3-channel 2D image (here 480x640x3) where each pixel of the 480x640 grid has 3 numbers specifying red, green, and blue intensities. In this case, these channels don't have unique information and we will just average over them and make this a single-channel image

In [ ]:
img = np.mean(img,axis=2)

Let's take a look at this image again, and this time with a different color palette

In [ ]:
plt.imshow(img,cmap=plt.cm.jet)

To analyze individual cells in the image, we need to pick them out of the image. This task is referred to as segmentation. We will utilize the cellpose python package, which is a pre-trained machine learning method utilizing a deep neural network and trained across many different kinds of cell imaging.

In [ ]:
!pip install cellpose #the exclamation point is a shorthand in a jupyter notebook (which google colab notebooks are built on) to run system commands
!pip install "opencv-python-headless<4.3"

To use cellpose, we need to specify a pre-trained neural network model. There are several models available, we will use one that is made to segment out individual cells from images of the cytoplasm, or cell body.

In [ ]:
from cellpose import models
model = models.Cellpose(model_type='cyto') #other model types are available, see cellpose documentation https://github.com/MouseLand/cellpose
model = models.Cellpose(model_type='cyto')

We can now run the model on our image, and see what the results look like-- this takes about a minute usually

In [ ]:
masks, flows, styles, diams = model.eval(img, diameter=None, flow_threshold=None, channels=[0,0]) #this command runs the pre-trained neural network model on our cell image

Cellpose provides a plotting tool to show us the output of the segmentation. For the provided default image (images/cell_image.jpg) the model picks out what looks to be many individual cells, as well as some of the debris.

In [ ]:
from cellpose import plot
fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, img, masks, flows[0], channels=[0,0])
plt.tight_layout()
plt.show()

What is the output? Lets focus on the masks-- these are a set of labels which "mask" over each individual object picked out by the segmentation.

In [ ]:
print(np.unique(masks))
ncells = np.max(masks) + 1

We can take a look at these labels one-by-one to see what they are

In [ ]:
plt.imshow(masks==10) #the logical 
plt.pause(.2)

Let's measure the cell areas in pixels. To do so, we will add up how many pixels are labeled with each unique label (integers up to ncells

In [ ]:
cell_sizes=np.zeros(ncells) #initialize array for output
for i in range(1,ncells):
    cell_sizes[i] = np.sum(masks==i) 

In [ ]:
hist = plt.hist(cell_sizes,bins=np.linspace(0,np.max(cell_sizes),20)) #create a histogram with 20 bins
plt.plot([np.mean(cell_sizes),np.mean(cell_sizes)],[0,np.max(hist[0])],'k--',label='average cell area')
plt.legend(loc='upper right')
plt.title('cell area histogram')
plt.ylabel('number of cells')
plt.xlabel('cell area (pixels)')

In [ ]:
np.max(hist[0])